
# AWS S3 Bucket Policy — Public Read Access (Console Lab)

### 🎯 Objective
Use the **AWS Management Console** to make an S3 object publicly readable **with a bucket policy**, and verify using the **object URL** in an incognito window. You will also learn all building blocks of a bucket policy and the interaction with **Block Public Access**.

> ⚠️ **Security note (for lab only):** Public buckets/objects are discouraged in production. Prefer **Amazon CloudFront with Origin Access Control (OAC)**, **pre‑signed URLs**, or **fine‑grained IAM**. Re‑enable *Block public access* and remove the policy when you finish.



## 🧠 Concepts You Must Know

**S3 access controls (four layers):**
- **Bucket policy** — JSON policy attached to a *bucket*. Great for global rules (allow/deny) that apply to many objects.
- **IAM identity policy** — permissions attached to users/roles (e.g., developers, applications).
- **ACLs** — legacy, object/bucket level grants. Avoid unless there is a special need.
- **Block public access** — _guardrail_ that can **override** any policy/ACL to prevent public exposure.

**Bucket policy building blocks:**
- `Version` — policy language version: `2012-10-17`.
- `Statement` — one or more entries:
  - `Sid` — optional identifier.
  - `Effect` — `Allow` or `Deny`.
  - `Principal` — who the statement applies to. `"*"` means “everyone” (the public internet).
  - `Action` — API(s) you allow/deny, e.g. `s3:GetObject` (read an object), `s3:ListBucket` (list object keys).
  - `Resource` — **ARN** of what you control.  
    - Bucket: `arn:aws:s3:::bucket-name`  
    - Objects: `arn:aws:s3:::bucket-name/*` (note the `/*` suffix for objects)
  - `Condition` — optional constraints (IP allowlist, VPC, referer, etc.).

**Common pitfalls**
- Forgetting `/*` in the object resource → `AccessDenied` for object reads.
- Leaving **Block public access** enabled → policy appears fine but still blocked.
- Allowing `ListBucket` when you only need `GetObject`.



## 🧰 Prerequisites

- AWS account with access to **S3**.
- Any sample file to upload, e.g. `developer.jpg` or `orders.csv`.
- Region of your choice (e.g. `us-east-1`).
- A browser (use **Incognito/Private** window for validation).



## Step 1 — Create a bucket and upload an object (private by default)

1. In the console, open **S3 → Buckets → Create bucket**.  
2. Bucket name must be globally unique, e.g. `bp-demo-<yourname>-<yyyyMMdd>`.
3. Keep **Block *all* public access** **enabled** (default).  
4. Create the bucket.
5. Open the bucket → **Upload** → add your file (e.g., `developer.jpg`).

Your bucket resembles this after upload:


![](sandbox:/mnt/data/520b7baf-bcfc-49a5-bbec-e9a775dfe6a3.png)


### Validate it’s private
- Click the object, copy the **Object URL**.
- Open the URL in an **Incognito** tab. You should get **AccessDenied** — that’s expected because the bucket is private.



## Step 2 — Understand & (temporarily) relax *Block public access*

S3 includes a guardrail called **Block public access**. If it is turned on, public ACLs and public bucket policies are **ignored**.

- Go to **Bucket → Permissions → Block public access (bucket settings) → Edit**.
- If you plan to apply a public bucket policy for this lab, **uncheck**:
  - *Block all public access* (or at minimum the policy-related checkboxes).  
- Save and confirm by typing **confirm** when prompted.

> Keep this off **only** for the duration of the lab; you will re-enable it in cleanup.

Screens from the console:


![](sandbox:/mnt/data/6b70462c-9e8e-4f52-9100-36243df973b3.png)

![](sandbox:/mnt/data/1f61447c-6f7d-4e95-9735-e30c7f53d1de.png)


## Step 3 — Open the **Bucket policy** editor

- Go to **Bucket → Permissions → Bucket policy → Edit**.

This is where we’ll paste JSON that allows the public to read objects:



![](sandbox:/mnt/data/de034917-24a2-41a7-b8f5-56de6e985550.png)


## Step 4 — Generate a minimal public‑read policy (using the Policy Generator)

1. Click **Policy generator** (opens in a new tab).  
2. **Select type of policy:** `S3 Bucket Policy`.  
3. **Effect:** `Allow`  
4. **Principal:** `*` (everyone)  
5. **AWS Service:** `Amazon S3`  
6. **Actions:** check **`GetObject`** (read objects).  
7. **Amazon Resource Name (ARN):** `arn:aws:s3:::<your-bucket-name>/*`  
   - The `/*` suffix means “all **objects** in this bucket.”  
8. Click **Add Statement**, then **Generate Policy** and copy the JSON.

Policy‑generator screens:


![](sandbox:/mnt/data/f6b8b4ae-c4e4-44e3-b13d-abf2cacd75ed.png)

![](sandbox:/mnt/data/1e591ce3-e5e7-4724-9ce1-11f08ea65b7d.png)

![](sandbox:/mnt/data/5406a66b-7e1d-4099-abcf-4cb949ff3d77.png)


### ✅ Minimal public‑read bucket policy (copy/paste)

Replace `<your-bucket-name>` and paste into the **Bucket policy** editor:

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AllowPublicRead",
      "Effect": "Allow",
      "Principal": "*",
      "Action": ["s3:GetObject"],
      "Resource": ["arn:aws:s3:::<your-bucket-name>/*"]
    }
  ]
}
```

> Why only `s3:GetObject`?  
> - It lets the public **download** objects if they know the exact key.  
> - It does **not** let the public list your bucket keys. (Listing would also require `s3:ListBucket` on the bucket ARN `arn:aws:s3:::<bucket>` — we are intentionally **not** granting that.)

After pasting, choose **Save changes**.

Editing and saving the policy example:


![](sandbox:/mnt/data/88dc51c8-29d4-43b2-bda9-ae6bbea104b2.png)


## Step 5 — Validate with the object URL (Incognito)

1. Go back to your object (e.g., `developer.jpg`) and copy the **Object URL**.
2. Open a **new incognito/private** browser tab and paste the URL.
3. You should now see the object load (image displays or file downloads).

Example of copying the object URL from the Properties tab:


![](sandbox:/mnt/data/46417997-d97e-4a65-b250-7900921fa185.png)


### Understanding S3 URL forms (for reference)
- **Virtual‑hosted‑style:** `https://<bucket>.s3.<region>.amazonaws.com/<key>`  
- **Path‑style (legacy):** `https://s3.<region>.amazonaws.com/<bucket>/<key>`  
- **Static website endpoint:** `http://<bucket>.s3-website-<region>.amazonaws.com/<key>` (separate feature; not required for this lab)



## Optional — Scope access to a folder/prefix only

If you only want to expose files under `public/`:

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AllowPublicReadUnderPrefix",
      "Effect": "Allow",
      "Principal": "*",
      "Action": ["s3:GetObject"],
      "Resource": ["arn:aws:s3:::<your-bucket-name>/public/*"]
    }
  ]
}
```



## Troubleshooting

- **Still AccessDenied?**  
  - Re‑check **Block public access** is **disabled** on the bucket during the lab.  
  - Ensure `Resource` uses `arn:aws:s3:::<bucket>/*` (with `/*`) for object reads.  
  - Confirm you pasted JSON into **Bucket policy**, not **CORS** or **ACLs**.  
  - If your org has SCPs/Service‑Control Policies, they can override your bucket policy.
- **Object URL 404 Not Found**  
  - The **key name** (path & case) must match exactly. Verify under the **Objects** tab.
- **Accidentally granted listing**  
  - If you added `s3:ListBucket` on `arn:aws:s3:::<bucket>`, the public can enumerate keys. Remove that statement unless you truly need it.



## 🧹 Cleanup (VERY IMPORTANT)

1. **Remove** the bucket policy (Permissions → Bucket policy → Delete and Save).  
2. **Re‑enable** **Block *all* public access** for the bucket.  
3. Delete the object(s) if you no longer need them.  
4. Optionally, delete the bucket.

This returns your environment to a secure default.



## 🧭 Reflection & Key Takeaways

| Topic | Insight |
|---|---|
| Bucket policy vs IAM policy | Bucket policies protect a **resource**; IAM policies grant rights to a **principal**. |
| `GetObject` vs `ListBucket` | Reading an object needs `GetObject` on `arn:aws:s3:::bucket/*`; listing keys needs `ListBucket` on `arn:aws:s3:::bucket`. |
| Block public access | Acts as a **master switch** to stop public exposure regardless of policies/ACLs. |
| Public access risks | Prefer CloudFront OAC, pre‑signed URLs, or VPC/private access for production. |
| Least privilege | Grant only the exact actions and resources required, and only for the duration needed. |

You have now created, explained, and validated a **bucket policy** for public reads — and you know how to roll it back safely.


![](sandbox:/mnt/data/75782226-dbdd-4aca-9318-48176b36405a.png)